# Data Conversion of SeapHOx V2 .hex data
This python notebook shows how to convert a .hex file into pandas dataframe with scientific values.

Please contact [SBS customer support](https://www.seabird.com/support) for help or to request additional features.

## Init

The initialization code section below is used to import required libraries.

In [ ]:
# Native imports
import os

# Third-party imports
import numpy as np
import pandas as pd

# Sea-Bird imports
import seabirdscientific.conversion as conv
import seabirdscientific.instrument_data as id
import seabirdscientific.visualization as viz
import example_data.example_coefficients as ec

## [Data Conversion](#proc-list)

This section shows how to convert raw data contained in a .hex file into scientific units for the instruments that follow:  
- SeapHOx V2

In [ ]:
hex_file = os.path.join("example_data", "SeapHOx_V2", "SeapHOx_V2.hex")

# Convert raw hexadecimal string to raw frequencies
raw_data = id.read_hex_file(
    filepath=hex_file, instrument_type=id.InstrumentType.SeapHox2, is_shallow=True
)

In [ ]:
# Convert raw values to scientific values

temperature_C = conv.convert_temperature(
    temperature_counts_in=raw_data["temperature"].values,
    coefs=ec.temperature_coefs_sn6385,
    standard="ITS90",
    units="C",
    use_mv_r=False,
)

pressure_dbar = conv.convert_pressure(
    pressure_count=raw_data["pressure"].values,
    compensation_voltage=raw_data["temperature compensation"],
    coefs=ec.pressure_coefs_sn6385,
    units="dbar",
)

conductivity = conv.convert_conductivity(
    conductivity_count=raw_data["conductivity"].values,
    temperature=temperature_C,
    pressure=pressure_dbar,  # must be in units of dbar
    coefs=ec.conductivity_coefs_sn6385,
)

# salinity = gsw.SP_from_C(
#     C=conductivity / 1000, # mS/cm
#     t=temperature, # ITS90 C
#     p=pressure_dbar # dbar
# )
salinity = np.full(len(temperature_C), 30.0)  # override until we have a real data set

ph_temperature = conv.convert_seafet_temperature(
    raw_temp=raw_data["ph temperature"].values, coefs=ec.temeprature_seafet_coefs
)

ph_internal = conv.convert_internal_seafet_ph(
    ph_counts=raw_data["vrs internal"].values,
    temperature=temperature_C,
    coefs=ec.ph_internal_coefs,
)

ph_external = conv.convert_external_seafet_ph(
    ph_counts=raw_data["vrs external"].values,
    temperature=ph_temperature,
    salinity=salinity,
    pressure=pressure_dbar,
    coefs=ec.ph_external_coefs,
)

# Calculating oxygen here in preliminary. Oxygen should be recalculated once pressure and salinity (both inputs to this calculation)
# are processed with the tools in processing.ipynb but before bin average.
oxygen = conv.convert_sbe63_oxygen(
    raw_oxygen_phase=raw_data["SBE63 oxygen phase"],
    thermistor=raw_data["SBE63 oxygen temperature"],
    pressure=pressure_dbar,
    salinity=salinity,
    coefs=ec.oxygen_63_coefs_sn2568,
    thermistor_coefs=ec.thermistor_63_coefs_sn2568,
    thermistor_units="volts",
)

# Flag to be used in data processing
flag = np.zeros(len(raw_data))

data = pd.DataFrame(
    {
        "temp": temperature_C,
        "cond": conductivity,
        "sal": salinity,
        "pres": pressure_dbar,
        "oxy": oxygen,
        "ph_internal": ph_internal,
        "ph_external": ph_external,
        "date_time": raw_data["date time"],
        "flag": flag,
    }
)

data

## [Data Plotting](#proc-list)

In [ ]:
config = viz.ChartConfig(
    title="SeaFET Data Conversion",
    x_names=["date_time"],
    y_names=["temp", "oxy", "ph_internal", "ph_external"],
    z_names=[],
    chart_type="overlay",
    plot_loop_edit_flags=False,
    lift_pen_over_bad_data=True,
)

chart_data = viz.ChartData(data, config)
fig = viz.plot_xy_chart(chart_data, config)

# plotly customizations
fig["layout"]["yaxis"]["autorange"] = "reversed"
fig.data[0].name = "Temperature"
fig.data[1].name = "Oxygen"
fig.data[2].name = "pH Internal"
fig.data[3].name = "pH External"
fig["layout"]["yaxis"]["title"] = "Temperature [ITS-90 C]"
fig["layout"]["yaxis2"]["title"] = "Oxygen [ml/l]"
fig["layout"]["yaxis3"]["title"] = "pH Internal"
fig["layout"]["yaxis4"]["title"] = "pH External"


fig.update_layout(height=800)
fig.show()

## Processing

This converted data should now be processed using the tools in processing.ipynb to produce a final data product.